## Task 2 - Machine Translation

- Import Libraries for Task 2

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive
%ls

In [4]:
%pip install datasets bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 92.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [5]:
# Libraries for 2C
from transformers import T5ForConditionalGeneration, T5Tokenizer
from datasets import load_dataset, load_metric
import datasets
import torch
from bert_score import score

# Libraries for 2A
import datasets
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from typing import Iterable, List

from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer

## Setup 2A

- Load train, val and test dataset

In [6]:
train_data = datasets.load_dataset("wmt16","de-en", split="train[:60000]")
val_data = datasets.load_dataset("wmt16","de-en", split="validation")
test_data = datasets.load_dataset("wmt16","de-en", split="test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/4548885 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2169 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2999 [00:00<?, ? examples/s]

In [ ]:
!python3 -m spacy download de_core_news_sm
!python3 -m spacy download en_core_web_sm

In [12]:
from torchtext.data.utils import get_tokenizer

tokenizer_german = get_tokenizer("spacy", language="de_core_news_sm")
tokenizer_english = get_tokenizer("spacy", language="en_core_web_sm")

- Mapping for tokenisation

In [13]:
# Set source and target languages
SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'

# Place-holders for the token transformers and vocabularies
token_transform = {SRC_LANGUAGE: tokenizer_german, TGT_LANGUAGE: tokenizer_english}
vocab_transform = {}

# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

# Helper function to yield list of tokens
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    for data_sample in data_iter:
        yield token_transform[language](data_sample['translation'][language])

# Build vocabularies for both languages
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    # Directly passing the dataset to yield_tokens without using .map
    vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train_data, ln),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)
    # Set UNK_IDX as the default index for vocabularies
    vocab_transform[ln].set_default_index(UNK_IDX)

In [14]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [15]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 32
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

# Load the transformer model from the saved state_dict "Setup2A_Model.pt"
transformer.load_state_dict(torch.load("Setup2A_Model.pt", map_location=DEVICE))

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


<All keys matched successfully>

- Helper Functions for Evaluation

In [16]:
from torch.nn.utils.rnn import pad_sequence

"""
Helper function to generate the masks
"""
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz))) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

"""
Helper function for performing greedy decoding
"""
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len - 1):
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys

"""
Function to translate a sentence using Greedy Decoding
"""
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    tokens = [BOS_IDX] + [vocab_transform[SRC_LANGUAGE][token] for token in tokenizer_german(src_sentence)] + [EOS_IDX]
    src = torch.tensor(tokens, dtype=torch.long).view(-1, 1).to(DEVICE)
    src_mask = (torch.zeros(len(tokens), len(tokens))).type(torch.bool).to(DEVICE)

    tgt_tokens = greedy_decode(
        model, src, src_mask, max_len=len(tokens) + 5, start_symbol=BOS_IDX).flatten()

    return " ".join([vocab_transform[TGT_LANGUAGE].lookup_token(token) for token in tgt_tokens.cpu().numpy() if token not in (BOS_IDX, EOS_IDX, PAD_IDX)])

- Inference on Validation and Test

In [17]:
# Evaluate the model on the val set
val_inputs = [d['translation'][SRC_LANGUAGE] for d in val_data]
val_targets = [d['translation'][TGT_LANGUAGE] for d in val_data]
val_predictions = []

for inp in val_inputs:
    val_predictions.append(translate(transformer, inp))

# Evaluate the model on the test set
test_inputs = [d['translation'][SRC_LANGUAGE] for d in test_data]
targets = [d['translation'][TGT_LANGUAGE] for d in test_data]
predictions = []

for inp in test_inputs:
    predictions.append(translate(transformer, inp))

- Computing metrics

In [21]:
from datasets import load_metric

# Convert targets to the expected format for METEOR (list of lists, one reference per prediction)
meteor_targets = [[target] for target in val_targets]

# Load METEOR and BLEU metrics
meteor_metric = load_metric('meteor', trust_remote_code=True)
bleu_metric = load_metric('bleu', trust_remote_code=True)

# Calculate METEOR score
meteor_results = meteor_metric.compute(predictions=val_predictions, references=meteor_targets)

# BLEU expects tokenized references and predictions, and references must be in a list of lists of lists
bleu_targets_tokenized = [[[word for word in tokenizer_english(target)] for target in [ref]] for ref in val_targets]
bleu_predictions_tokenized = [[word for word in tokenizer_english(pred)] for pred in val_predictions]

# Calculate BLEU score
bleu_results_1 = bleu_metric.compute(predictions=bleu_predictions_tokenized, references=bleu_targets_tokenized, max_order=1)
bleu_results_2 = bleu_metric.compute(predictions=bleu_predictions_tokenized, references=bleu_targets_tokenized, max_order=2)
bleu_results_3 = bleu_metric.compute(predictions=bleu_predictions_tokenized, references=bleu_targets_tokenized, max_order=3)
bleu_results_4 = bleu_metric.compute(predictions=bleu_predictions_tokenized, references=bleu_targets_tokenized, max_order=4)

# Calculate BERTScore
from bert_score import score

P, R, F1 = score(val_predictions, val_targets, lang='en', verbose=True)

print("VALIDATION DATA: ")
print()
print("METEOR Score:", meteor_results["meteor"])
print()
print("BLEU-1 Score:", bleu_results_1["bleu"])
print("BLEU-2 Score:", bleu_results_2["bleu"])
print("BLEU-3 Score:", bleu_results_3["bleu"])
print("BLEU-4 Score:", bleu_results_4["bleu"])
print()
print(f"BERTScore-F1: {F1.mean()}")
print(f"BERTScore-Precision: {P.mean()}")
print(f"BERTScore-Recall: {R.mean()}")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/68 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/34 [00:00<?, ?it/s]

done in 24.43 seconds, 88.79 sentences/sec
VALIDATION DATA: 

METEOR Score: 0.07892144841864594

BLEU-1 Score: 0.14692474928115573
BLEU-2 Score: 0.05172191734783846
BLEU-3 Score: 0.01575246806972898
BLEU-4 Score: 0.005401003409349793

BERTScore-F1: 0.7921583652496338
BERTScore-Precision: 0.7647544741630554
BERTScore-Recall: 0.8221092820167542


In [20]:
from datasets import load_metric

# Convert targets to the expected format for METEOR (list of lists, one reference per prediction)
meteor_targets = [[target] for target in targets]

# Load METEOR and BLEU metrics
meteor_metric = load_metric('meteor', trust_remote_code=True)
bleu_metric = load_metric('bleu', trust_remote_code=True)

# Calculate METEOR score
meteor_results = meteor_metric.compute(predictions=predictions, references=meteor_targets)

# BLEU expects tokenized references and predictions, and references must be in a list of lists of lists
bleu_targets_tokenized = [[[word for word in tokenizer_english(target)] for target in [ref]] for ref in targets]
bleu_predictions_tokenized = [[word for word in tokenizer_english(pred)] for pred in predictions]

# Calculate BLEU score
bleu_results_1 = bleu_metric.compute(predictions=bleu_predictions_tokenized, references=bleu_targets_tokenized, max_order=1)
bleu_results_2 = bleu_metric.compute(predictions=bleu_predictions_tokenized, references=bleu_targets_tokenized, max_order=2)
bleu_results_3 = bleu_metric.compute(predictions=bleu_predictions_tokenized, references=bleu_targets_tokenized, max_order=3)
bleu_results_4 = bleu_metric.compute(predictions=bleu_predictions_tokenized, references=bleu_targets_tokenized, max_order=4)

# Calculate BERTScore
from bert_score import score

P, R, F1 = score(predictions, targets, lang='en', verbose=True)

print("TEST DATA: ")
print()
print("METEOR Score:", meteor_results["meteor"])
print()
print("BLEU-1 Score:", bleu_results_1["bleu"])
print("BLEU-2 Score:", bleu_results_2["bleu"])
print("BLEU-3 Score:", bleu_results_3["bleu"])
print("BLEU-4 Score:", bleu_results_4["bleu"])
print()
print(f"BERTScore-F1: {F1.mean()}")
print(f"BERTScore-Precision: {P.mean()}")
print(f"BERTScore-Recall: {R.mean()}")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/94 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/47 [00:00<?, ?it/s]

done in 32.75 seconds, 91.56 sentences/sec
TEST DATA: 

METEOR Score: 0.0731069342940759

BLEU-1 Score: 0.13277656590838263
BLEU-2 Score: 0.04483180258732872
BLEU-3 Score: 0.013891481438070667
BLEU-4 Score: 0.004510730766538279

BERTScore-F1: 0.7899057269096375
BERTScore-Precision: 0.7632051706314087
BERTScore-Recall: 0.8191089630126953


## Setup 2C

- Extract Model 2C

In [22]:
import zipfile
import os

# If the file is in Google Drive, adjust the path accordingly
zip_path = '/content/drive/MyDrive/Setup2C_Model.zip'

extract_to_path = '/content/'

# Ensure the extract_to_path directory exists
if not os.path.exists(extract_to_path):
    os.makedirs(extract_to_path)

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

print(f'Extracted to: {extract_to_path}')

Extracted to: /content/


In [23]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the model
model = T5ForConditionalGeneration.from_pretrained('/content/Setup2C_Model/t5-small-de-en-model')

# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained('/content/Setup2C_Model/t5-small-de-en-tokenizer')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [24]:
validation_dataset = datasets.load_dataset("wmt16","de-en", split="validation")
test_dataset = datasets.load_dataset("wmt16","de-en", split="test")

In [ ]:
def evaluate_model(dataset):
    predictions, references = [], []
    for example in dataset:
        input_ids = tokenizer("translate German to English: " + example["translation"]["de"], return_tensors="pt").input_ids.to(model.device)
        output = model.generate(input_ids, max_new_tokens=128)
        pred_text = tokenizer.decode(output[0], skip_special_tokens=True)
        predictions.append(pred_text)
        references.append(example["translation"]["en"])

    # Load METEOR and BLEU metrics
    meteor_metric = load_metric('meteor', trust_remote_code=True)
    bleu_metric = load_metric('bleu', trust_remote_code=True)

    # Convert targets to the expected format for METEOR (list of lists, one reference per prediction)
    meteor_targets = [[target] for target in references]

    # Calculate METEOR score
    meteor_results = meteor_metric.compute(predictions=predictions, references=meteor_targets)

    # BLEU expects tokenized references and predictions, and references must be in a list of lists of lists
    bleu_targets_tokenized = [[[word for word in target.strip().split()] for target in [ref]] for ref in references]
    bleu_predictions_tokenized = [[word for word in pred.strip().split()] for pred in predictions]

    # Calculate BLEU score
    bleu_results_1 = bleu_metric.compute(predictions=bleu_predictions_tokenized, references=bleu_targets_tokenized, max_order=1)
    bleu_results_2 = bleu_metric.compute(predictions=bleu_predictions_tokenized, references=bleu_targets_tokenized, max_order=2)
    bleu_results_3 = bleu_metric.compute(predictions=bleu_predictions_tokenized, references=bleu_targets_tokenized, max_order=3)
    bleu_results_4 = bleu_metric.compute(predictions=bleu_predictions_tokenized, references=bleu_targets_tokenized, max_order=4)

    # Calculate BERTScore
    from bert_score import score

    P, R, F1 = score(predictions, references, lang='en', verbose=True)

    return bleu_results_1['bleu'], bleu_results_2['bleu'], bleu_results_3['bleu'], bleu_results_4['bleu'], meteor_results['meteor'], P, R, F1

bleu1, bleu2, bleu3, bleu4, meteor, bert_p, bert_r, bert_f1 = evaluate_model(validation_dataset)

In [27]:
print("VALIDATION DATA:")
print()
print("METEOR Score:", meteor)
print()
print("BLEU-1 Score:", bleu1)
print("BLEU-2 Score:", bleu2)
print("BLEU-3 Score:", bleu3)
print("BLEU-4 Score:", bleu4)
print()
print(f"BERTScore-F1: {bert_f1.mean()}")
print(f"BERTScore-Precision: {bert_p.mean()}")
print(f"BERTScore-Recall: {bert_r.mean()}")

VALIDATION DATA:

METEOR Score: 0.35757361250587055

BLEU-1 Score: 0.4091096805538686
BLEU-2 Score: 0.26456833610344566
BLEU-3 Score: 0.17844615973866995
BLEU-4 Score: 0.12460686818331085

BERTScore-F1: 0.9052720069885254
BERTScore-Precision: 0.9027578830718994
BERTScore-Recall: 0.908021092414856


In [ ]:
bleu1, bleu2, bleu3, bleu4, meteor, bert_p, bert_r, bert_f1 = evaluate_model(test_dataset)

print("TEST DATA: ")
print()
print("METEOR Score:", meteor)
print()
print("BLEU-1 Score:", bleu1)
print("BLEU-2 Score:", bleu2)
print("BLEU-3 Score:", bleu3)
print("BLEU-4 Score:", bleu4)
print()
print(f"BERTScore-F1: {bert_f1.mean()}")
print(f"BERTScore-Precision: {bert_p.mean()}")
print(f"BERTScore-Recall: {bert_r.mean()}")

## Inference Pipeline for Task 2

In [30]:
def translate_to_english_2A(inp):
  translated_sentence = translate(transformer, inp)
  return translated_sentence

In [32]:
def translate_to_english_2C(inp):
  input_ids = tokenizer("translate German to English: " + inp, return_tensors="pt").input_ids.to(model.device)
  output = model.generate(input_ids, max_new_tokens=128)
  pred_text = tokenizer.decode(output[0], skip_special_tokens=True)
  return pred_text

In [33]:
translate_to_english_2A("Das Leben ist wie eine Schachtel Pralinen, man weiß nie, was man bekommt.")

', as well , know , what is what you know , and , as well , how , you know ,'

In [34]:
translate_to_english_2C("Das Leben ist wie eine Schachtel Pralinen, man weiß nie, was man bekommt.")

'The life is like a shamble pralinen, who never knows what he gets.'

### 2A

In [29]:
import pandas as pd

#Load the CSV file
input_df = pd.read_csv('input.csv')

input_df['en'] = input_df['de'].apply(translate_to_english_2A)

input_df.to_csv('translated_output_2A.csv', index=False)


### 2C

In [ ]:
input_df2 = pd.read_csv('input.csv')

input_df2['en'] = input_df2['de'].apply(translate_to_english_2A)

input_df.to_csv('translated_output_2C.csv', index=False)